In [7]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import tkinter as tk
from tkinter import ttk
from fuzzywuzzy import fuzz
import time
import pygame
import io
import requests
import numpy as np
from PIL import Image, ImageTk


# read the client_id, client_secret, and redirect_uri from the file
with open('spotify_credentials.txt', 'r') as f:
    client_id = f.readline().strip()
    client_secret = f.readline().strip()
    redirect_uri = f.readline().strip()
    

# Authenticate with Spotify
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri=redirect_uri,
                                               scope='user-read-private user-read-playback-state user-modify-playback-state'))

# Fetch user playlists
def fetch_playlists():
    playlists = sp.current_user_playlists()
    return [playlist['name'] for playlist in playlists['items']]

playlist_names = fetch_playlists()

In [14]:
root = tk.Tk()
root.title("Spotify Guess the Artist Game")
root.geometry("1200x600")
root.configure(bg='#191414')

# apply circular std font to all widgets
root.option_add('*Font', 'CircularStd-Bold 16')
#make font bold
# Update the background color of all widgets to green and the text color to black.
root.option_add('*background', '#191414')
root.option_add('*foreground', '#1DB954')  # Set text color to white using hex code

# Define fonts and colors
title_font = ("Helvetica", 16, "bold")
button_font = ("Helvetica", 12)
label_font = ("Helvetica", 12)
bg_color = "#f0f0f0"
button_color = "#1DB954"  # Spotify green
button_fg_color = "#ffffff"
dropdown_bg_color = "#191414"  # Spotify black
dropdown_fg_color = "#ffffff"


songs_list = None
current_track = None
correct_answers = 0
incorrect_answers = 0
total_attempts = 0
songs_skipped = 0
total_songs = 0
first_time_guess = 0
incorrect_temp = 0

def fetch_playlists():
    playlists = sp.current_user_playlists()
    return {playlist['name']: playlist['id'] for playlist in playlists['items']}

def fetch_tracks(playlist_id):
    tracks = sp.playlist_tracks(playlist_id)
    return {track['track']['name']: track['track']['id'] for track in tracks['items']}

def restart_song():
    sp.start_playback()

def play_song(track_id):
    device_id = get_active_device()
    if device_id:
        sp.start_playback(device_id=device_id, uris=[f"spotify:track:{track_id}"])
    else:
        print("No active device found. Please open Spotify on a device.")

def stop_song():
    sp.pause_playback()

def restart_song():
    sp.start_playback()

def normalize_string(s):
    """
    Normalize the string by converting it to lowercase and stripping whitespace.
    """
    return s.strip().lower()

def compare_strings_fuzzy(provided_string, given_string, threshold=85):
    """
    Compare the provided string with the given string using fuzzy matching.
    Return True if the similarity score is above the threshold, otherwise False.
    """
    normalized_provided = normalize_string(provided_string)
    normalized_given = normalize_string(given_string)
    similarity_score = fuzz.ratio(normalized_provided, normalized_given)
    return similarity_score >= threshold

def unpack_combobox(combobox):
    combobox.pack_forget()

def on_combobox_select(event):
    global songs_list
    songs_list = None
    songs_list = get_track_ids(playlist_var.get())
    unpack_combobox(playlist_dropdown)
    start_game_button.pack()

def get_active_device():
    devices = sp.devices()
    if devices['devices']:
        return devices['devices'][0]['id']
    else:
        return None

def get_track_ids(playlist_name):
    playlist_id = playlist_dict[playlist_name]
    tracks = fetch_tracks(playlist_id)
    return list(tracks.values())

def resume_song():
    sp.start_playback()

def reset_song():
    sp.seek_track(0)

def pause_song():
    sp.pause_playback()

def fetch_artists(current_track):
    track = sp.track(current_track)
    return [artist['name'] for artist in track['artists']]

def fetch_current_track():
    return sp.current_playback()['item']['id']

def check_answer():
    global current_track
    global correct_answers
    global total_attempts
    global songs_skipped
    global first_time_guess
    global incorrect_temp
    global incorrect_answers

    current_track = fetch_current_track()
    guessed_artist = guess_entry.get().strip()
    actual_artists = fetch_artists(current_track)
    # write a condititon that if any of element of list is True then return True
    if any(compare_strings_fuzzy(guessed_artist, artist) for artist in actual_artists):
        guess_label.forget()
        result_label.config(text="Correct!")
        result_label.pack()
        result_label.after(2000, lambda: result_label.pack_forget())
        guess_entry.delete(0, tk.END)
        pause_song()
        pause_button.forget()
        resume_button.forget()
        restart_button.forget()
        surrender_button.forget()
        guess_entry.forget()
        check_button.forget()
        display_song_details()
        correct_answers += 1
        total_attempts += 1
        if incorrect_temp == 0:
            first_time_guess += 1
        update_stats_label()
        play_next_song_button.pack()
        stats_label.pack()
        play_preview()
        

    else:
        result_label.config(text="Incorrect. Try again!")
        guess_entry.delete(0, tk.END)
        result_label.config(text="Incorrect!")
        result_label.pack()
        result_label.after(2000, lambda: result_label.pack_forget())
        total_attempts += 1
        incorrect_answers += 1
        incorrect_temp += 1
        update_stats_label()

def update_stats_label():
    global stats_label
    stats_label.config(text=f"Your score is currently {correct_answers}/{total_attempts}\nYou have skipped {songs_skipped} songs.")

def display_song_details():
    """
    Fetches and displays the details of the current track being played on Spotify.

    This function retrieves the current track's details including the song name, artist(s), album name, 
    year of release, and album cover image. It then updates the GUI to display these details.

    Global Variables:
    - current_track: A dictionary containing details of the current track.
    - album_cover_label: A Tkinter Label widget to display the album cover image.
    - song_details_label: A Tkinter Label widget to display the song details.

    Fetches:
    - Current track details using `sp.track(fetch_current_track())`.
    - Artist names using `fetch_artists(fetch_current_track())`.
    - Album cover image from the URL provided in the track details.

    Updates:
    - `song_details_label` with the song name, artist(s), album name, and year of release.
    - `album_cover_label` with the album cover image.
    """
    global current_track
    global album_cover_label
    global song_details_label
    song_name = sp.track(current_track)['name']
    artist_name = fetch_artists(current_track)
    album_name = sp.track(current_track)['album']['name']
    year_of_release = sp.track(current_track)['album']['release_date'][:4]
    album_cover_url = sp.track(current_track)['album']['images'][0]['url']
    image_bytes = requests.get(album_cover_url).content
    image = Image.open(io.BytesIO(image_bytes))
    image = image.resize((200, 200))
    image_tk = ImageTk.PhotoImage(image)
    song_details_label = tk.Label(root, text=f"Song: {song_name}\nArtist(s): {', '.join(artist_name)}\nAlbum: {album_name}\nYear of release: {year_of_release}\n", font='CircularStd-Bold 16')
    song_details_label.pack()
    album_cover_label = tk.Label(root, image=image_tk)
    album_cover_label.image = image_tk
    album_cover_label.pack()

def next_song_action():
    """
    Handles the transition to the next song in the application.

    This function performs the following actions:
    1. Hides the current album cover, song details, and the "play next song" button.
    2. Plays the next song in the playlist.
    3. Displays the pause button, restart button, guess label, guess entry field, and check button.
    """
    album_cover_label.forget()
    song_details_label.forget()
    play_next_song_button.forget()
    stats_label.forget()
    time.sleep(1)
    play_random_song()
    pause_button.pack()
    restart_button.pack()
    guess_label.pack()
    guess_entry.pack()
    check_button.pack()
    surrender_button.pack()
    stop_preview()

def start_button_action():
    """
    Handles the action when the start button is pressed.
 
    """
    
    start_game_button.forget()
    time.sleep(1.5)
    play_random_song()
    pause_button.pack()
    restart_button.pack()
    surrender_button.pack()
    guess_label.pack()
    guess_entry.pack()
    check_button.pack()
    
def song_details_button_action():
    display_song_details()

def pause_button_action():
    pause_song()
    pause_button.forget()
    resume_button.pack()

def resume_button_action():
    resume_song()
    resume_button.forget()
    pause_button.pack()

def restart_button_action():
    reset_song()

def surrender_button_action():
        global current_track
        global correct_answers
        global total_attempts
        global songs_skipped
        current_track = fetch_current_track()
        #guess_label.forget()
        result_label.config(text="Song forfeited!")
        result_label.pack()
        result_label.after(2000, lambda: result_label.pack_forget())
        guess_entry.delete(0, tk.END)
        pause_song()
        pause_button.forget()
        resume_button.forget()
        restart_button.forget()
        surrender_button.forget()
        guess_label.forget()
        guess_entry.forget()
        check_button.forget()
        display_song_details()
        #count_songs_skipped()
        songs_skipped += 1
        update_stats_label()
        stats_label.pack()
        play_next_song_button.pack()
        play_preview()
        
def play_preview():
    """
    Play the 30-second MP3 preview of the song using pygame.
    """
    global current_track
    preview_url = sp.track(current_track)['preview_url']
    if preview_url:
        response = requests.get(preview_url)
        audio_data = io.BytesIO(response.content)
        
        pygame.mixer.init()
        pygame.mixer.music.load(audio_data)
        pygame.mixer.music.play()

        # Stop the preview after 10 seconds
        #root.after(30000, stop_preview)
    else:
        print("No preview available for this track.")

def stop_preview():
    pygame.mixer.music.stop()

def play_random_song():
    global songs_list
    global current_track
    global total_songs
    global incorrect_temp
    if songs_list:
        random_index = np.random.randint(len(songs_list))
        random_song = songs_list.pop(random_index)
        play_song(random_song)
        total_songs += 1
        incorrect_temp = 0
    else:
        print("No more songs to play.")

def update_stats_label():
    global stats_var
    global correct_answers
    global total_attempts
    global songs_skipped
    global first_time_guess

    precision = correct_answers / total_attempts if total_attempts > 0 else 0
    stats_var.set(f"You correctly guessed {correct_answers} ({first_time_guess} with first attempt), you were wrong {incorrect_answers} times and skipped {songs_skipped}. Your precision is {precision:.2f}")

# create a dropdown list of playlists, once a playlist is selected, the buttons will be unpacked
playlist_dict = fetch_playlists()




style = ttk.Style()
style.theme_use('clam')
style.configure("TCombobox", fieldbackground=dropdown_bg_color, background=dropdown_bg_color, foreground=dropdown_fg_color)
# Buttons

start_game_button = tk.Button(root, text="Start Game", command = lambda: start_button_action())
song_details_button = tk.Button(root, text="Song Details", command = lambda: song_details_button_action())
pause_button = tk.Button(root, text="Pause", command = lambda:  pause_button_action())
resume_button = tk.Button(root, text="Resume", command =  lambda:  resume_button_action())
restart_button = tk.Button(root, text="Restart", command = lambda:  restart_button_action())
surrender_button = tk.Button(root, text="Surrender", command = lambda: surrender_button_action())
play_next_song_button = tk.Button(root, text="Play Next Song", command = lambda: next_song_action())
result_label = tk.Label(root, text="")
playlist_var = tk.StringVar()
playlist_dropdown = ttk.Combobox(root, textvariable=playlist_var, style="TCombobox")
playlist_dropdown.place(relx=0.5, rely=0.5, anchor=tk.CENTER)
playlist_dropdown.set("Select Playlist")
playlist_dropdown['values'] = playlist_names
# place playlist dropdown in the middle on the gui and keep it constant



guess_label = tk.Label(root, text="Guess the Artist:")
guess_entry = tk.Entry(root)
check_button = tk.Button(root, text="Check Answer", command = lambda: check_answer())
# create label to display statistics in a manner "Your score is currently correct_answers/total_attempts", "You have skipped songs_skipped songs".
stats_var = tk.StringVar(value=f"Your score is currently {correct_answers}/{total_attempts}\nYou have skipped {songs_skipped} songs.")
stats_label = tk.Label(root, textvariable=stats_var)



    
playlist_dropdown.bind("<<ComboboxSelected>>", on_combobox_select)


playlist_dropdown.pack()
root.mainloop() 